In [1]:
import os
from datetime import datetime

## Define input and output filepaths

In [2]:
MATTERPORT_DIR = "/media/rsl_admin/T7/matterport/data/v1/scans"

LATTICE_GRAPHS_DIR = "/media/rsl_admin/T7/matterport/lattice_graphs"

In [3]:
OPENSCENE_REGION_SEGS_DIR = \
    "/home/rsl_admin/openscene/comparison_outputs/segmentations/object-openseg-matterport-test"


In [4]:
OUTPUT_DIR = "/home/rsl_admin/openscene/comparison_outputs/eval_outputs"

save_dir = os.path.join(OUTPUT_DIR, f"object-{datetime.now().strftime('%Y-%m-%d_%H-%M')}")
os.makedirs(save_dir, exist_ok=True)

print(f"saving outputs to: {save_dir}")

saving outputs to: /home/rsl_admin/openscene/comparison_outputs/eval_outputs/object-2024-05-18_23-11


## Figure out which scans to process

In [5]:
scans = set()
for fname in os.listdir(OPENSCENE_REGION_SEGS_DIR):
    scans.add(fname.split("_")[0])
    
scans = tuple(sorted(scans))

print(scans)

('2t7WUuJeko7', '5ZKStnWn8Zo', 'ARNzJeq3xxb', 'RPmz2sHmrrY', 'UwV83HsGsw3', 'Vt2qJdWjCF2', 'WYY7iVyf5p8', 'YFuZgdQ5vWj', 'YVUC4YcDtcY', 'fzynW3qQPVF', 'gYvKGZ5eRqb', 'gxdoqLR6rwA', 'jtcxE69GiFV', 'pa4otMbVnkk', 'q9vSo1VnCiC', 'rqfALeAoiTq', 'wc2JMjhGNzB', 'yqstnuAEVhm')


## Evaluation parameters

In [6]:
BLACKLISTED_OBJECT_LABELS = (
    "misc", "objects", "void", "unlabeled",
    "wall", "floor", "ceiling", "other"
)

In [7]:
params = {
        "label_params": {
            "blacklisted_labels": BLACKLISTED_OBJECT_LABELS,
        },
    
        # Make sure sides of proposal boxes are at least this long
        "min_proposal_box_length": 0.1,
    
    
        # From email with the author of OVIR
        "cluster_min_points": 50,

        # https://github.com/search?q=repo%3Ashiyoung77%2FOVIR-3D%20dbscan&type=code
        "dbscan_params": {
            "eps": 0.1,
            "min_points": 1,
            "print_progress": False,
        },
    }

## Run evaluation on the scans in the segmentations directory

In [8]:
from evaluate_matterport_scan_object import evaluate_matterport_scan_object_localizations

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [9]:
for scan_name in scans:

    scan_dir = os.path.join(MATTERPORT_DIR, f"{scan_name}/{scan_name}")
    lattice_graph_path = os.path.join(LATTICE_GRAPHS_DIR, f"{scan_name}_lattice_graph.pkl")
    
    evaluate_matterport_scan_object_localizations(
        params=params,
        scan_dir=scan_dir,
        openscene_region_segs_dir=OPENSCENE_REGION_SEGS_DIR,
        lattice_graph_path=lattice_graph_path,
        output_dir=save_dir
    )

running evaluation on matterport scan 2t7WUuJeko7
loaded house file from /media/rsl_admin/T7/matterport/data/v1/scans/2t7WUuJeko7/2t7WUuJeko7/house_segmentations/2t7WUuJeko7.house
loaded ply file from /media/rsl_admin/T7/matterport/data/v1/scans/2t7WUuJeko7/2t7WUuJeko7/house_segmentations/2t7WUuJeko7.ply
found the following region segmentation files for scan 2t7WUuJeko7:
2t7WUuJeko7_region4.pkl
	2t7WUuJeko7_region3.pkl
	2t7WUuJeko7_region5.pkl
	2t7WUuJeko7_region1.pkl
	2t7WUuJeko7_region0.pkl
	2t7WUuJeko7_region2.pkl


[warning]: bounding box with invalid category_index -1


loaded lattice graph from /media/rsl_admin/T7/matterport/lattice_graphs/2t7WUuJeko7_lattice_graph.pkl
aggregated region segmentations
started localization pipeline
100%|████████████████████████████████████████████████████████████████| 32/32 [00:02<00:00, 14.44it/s]
finished running localization pipeline
/home/rsl_admin/openscene/openscene_env/lib/python3.8/site-packages/cvxpy/reductions/solvers/solving_chain.py:354: FutureWarning: 
    You specified your problem should be solved by ECOS. Starting in
    CXVPY 1.6.0, ECOS will no longer be installed by default with CVXPY.
    Please either add an explicit dependency on ECOS or switch to our new
    default solver, Clarabel, by either not specifying a solver argument
    or specifying ``solver=cp.CLARABEL``.
    
  warnings.warn(ECOS_DEP_DEPRECATION_MSG, FutureWarning)
started computing precision metrics
  0%|                                                                         | 0/18 [00:00<?, ?it/s]proposal with no assigned lattice 

proposal with no assigned lattice nodes in towel, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in towel, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in towel, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in cushion, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in cushion, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in cushion, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in cushion, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in cushion, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in cushion, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in cushion, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in cus

[warning]: bounding box with invalid category_index -1


...

proposal with no assigned lattice nodes in seating, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in seating, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in seating, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in seating, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in seating, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in seating, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in seating, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in seating, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in seating, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in seating, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes 

proposal with no assigned lattice nodes in shower, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in shower, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in shower, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in shower, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in shower, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in shower, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in shower, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in shower, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in shower, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in shower, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in shower,

proposal with no assigned lattice nodes in shower, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in shower, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in shower, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in shower, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in shower, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in shower, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in shower, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in shower, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in shower, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in shower, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in shower,

proposal with no assigned lattice nodes in counter, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in counter, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in counter, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in counter, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in counter, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in counter, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in counter, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in counter, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in counter, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in counter, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes 

proposal with no assigned lattice nodes in counter, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in counter, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in counter, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in counter, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in counter, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in counter, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in counter, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in counter, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in counter, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in counter, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes 